In [ ]:
# https://medium.com/@prabhnoor0212/siamese-network-keras-31a3a8f37d04
# https://github.com/prabhnoor0212/Siamese-Network-Text-Similarity/blob/master/quora_siamese.ipynb

In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 13.6MB/s 
     |████████████████████████████████| 890kB 23.0MB/s 
     |████████████████████████████████| 3.0MB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c1a20b1f08a4db1679218603f06f1d97861195396f7681cac8dad39d6c69bd4e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
def get_input_ids(sentences, tokenizer):
  input_ids = []
  for sent in sentences:
      # `encode` will:
      #   (1) Tokenize the sentence.(2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.(4) Map tokens to their IDs.
      encoded_sent = tokenizer.encode(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          # This function also supports truncation and conversion
                          # to pytorch tensors, but we need to do padding, so we
                          # can't use these features.
                          # max_length = 128,          # Truncate all sentences.
                          # return_tensors = 'pt',     # Return pytorch tensors.
                    )
      input_ids.append(encoded_sent)
  return input_ids

from keras.preprocessing.sequence import pad_sequences

def get_padded_Sentences(sentences, tokenizer, length_long_sentence): # to have all sentences of same length
  # print("len of sentences:"+ str(len(sentences)))
  input_ids = []
  input_ids = get_input_ids(sentences, tokenizer) 
  # print(len(input_ids))
  # length_long_sentence = 45 # same as the max length used while initialising the model
  # print('Max sentence length: ', max([len(sen) for sen in input_ids]))
  padded_sentences = pad_sequences(input_ids, length_long_sentence, padding='post')
  # print(padded_sentences)
  return padded_sentences

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
print('Loaded BERT tokenizer.')
vocab_tokens = list(tokenizer.vocab.keys())
print("Original Vocab size: " + str(len(vocab_tokens))) 


Loaded BERT tokenizer.
Original Vocab size: 30522


In [ ]:
## To upload files from Local
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
import pandas as pd
import numpy as np

df_2 = pd.read_csv("./2_man_ann_sb.csv",  index_col= None)
df_2 = df_2.dropna(subset = ['tweet_text'])

df_1 = pd.read_csv("./mann_ann_sb.csv", index_col= None)

df_3 = pd.read_csv("./3_man_ann_sb_full_1.csv", index_col = None)
df_3 = df_3.dropna(subset = ['tweet_text'])

df_raw = df_1.append(df_2).append(df_3) # using batch 1 and batch 2 for training

all_tweets = df_1.append(df_2).append(df_3)
print(all_tweets.shape)
df_raw.shape

(7500, 19)


(7500, 19)

In [ ]:
# Remove ads marked as below because they are not available in ad annotations file although tweets mention them
# commercials, joe biden, pizzahut, joe bieden, michael bloomberg, mike bloomberg, scientology
ads_remove = ['commercials', 'joe biden', 'pizzahut', 'joe bieden', 'michael bloomberg', 'mike bloomberg', 'scientology']
# ads_rename = ['nfl100', 'tide']
df_raw['ad_manual_adjusted'] = df_raw['ad_manual_adjusted'].apply(lambda x: x.lower())
print(df_raw.shape)
df = pd.DataFrame()
removed_Data = pd.DataFrame()

for i,row  in df_raw.iterrows():
  if row['ad_manual_adjusted'] not in ads_remove:
    df = df.append(row)
  else:
    removed_Data = removed_Data.append(row)
print(df.shape)

(7500, 19)
(7406, 19)


In [ ]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

# stop = stopwords.words('english')
stop = []
# stop.append('superbowl')
# stop.append('super')
# stop.append('bowl')
# stop.append('commercial')
# stop.append('ad')
# stop.append('commercials')
# stop.append('ads')
# stop.extend(remaining_words)
print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '-'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&', '-', '_']
    text = "".join((char for char in text if char not in punctuation))
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 

#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0
cadillacabc joinrt the k jamesbond


In [ ]:
df['text_clean'] = df['tweet_text'].apply(lambda x: cleanTweet(x))
df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
df['ad_related'] = df['ad_manual_adjusted'].apply(lambda x: 0 if x == 'none' else 1)

filter_comma_ads = True
if filter_comma_ads:
  df = df[~df['ad_manual_adjusted'].str.contains(",")]
print(df.shape)

(7344, 22)


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter

allWords = []
for tweet in df['text_clean']:
  allWords.extend(nltk.tokenize.word_tokenize(tweet))

allWordDist = nltk.FreqDist(w.lower() for w in allWords)

stopwords = nltk.corpus.stopwords.words('english')
allWordExceptStopDist = nltk.FreqDist(w.lower() for w in allWords if w not in stopwords)   

print('size of corpus after removing stop words: '+ str(len(allWordExceptStopDist)))
mostCommon = allWordDist.most_common(5000) # returns a list of tuples
print('Length of top n most common words: '+ str(len(mostCommon)))
print(mostCommon)
mostCommonWords = [a_tuple[0] for a_tuple in mostCommon]
print(mostCommonWords)

remaining_words = []
print(allWordExceptStopDist)
for word in allWordExceptStopDist:
  if word not in mostCommonWords:
    remaining_words.append(word)

print(len(remaining_words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
size of corpus after removing stop words: 11002
Length of top n most common words: 5000
[('the', 4827), ('a', 2432), ('to', 2183), ('i', 1635), ('and', 1631), ('in', 1512), ('that', 1335), ('is', 1233), ('for', 1210), ('of', 1188), ('this', 1012), ('you', 927), ('was', 897), ('ers', 892), ('s', 863), ('it', 854), ('on', 721), ('trump', 643), ('with', 560), ('chiefs', 544), ('my', 520), ('we', 461), ('t', 432), ('me', 432), ('so', 414), ('be', 408), ('nfl', 408), ('just', 396), ('chiefskingdom', 394), ('all', 393), ('he', 390), ('but', 389), ('are', 381), ('at', 377), ('not', 373), ('one', 349), ('game', 326), ('have', 316), ('they', 305), ('from', 300), ('best', 297), ('an', 295), ('show', 293), ('brady', 290), ('what', 289), ('shakira', 283), ('great', 278), ('d',

In [ ]:
def get_ad_related_twts(df, removeCommas = True):
  df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
  ad_filter = df['ad_manual_adjusted']!= 'none'
  ad_related_twts = df[ad_filter]
  if removeCommas:
    ad_filter_1 = ~ad_related_twts['ad_manual_adjusted'].str.contains(',')
    ad_related_twts = ad_related_twts[ad_filter_1]
  return ad_related_twts

def getAdTweets(ad_related_twts, ad):
  return ad_related_twts[ad_related_twts.ad_manual_adjusted == ad].shape[0]

def get_ad_id_dict(ad_related_twts): 
  n_ad_related = ad_related_twts.shape[0]
  print("# ad related tweets: "+ str(n_ad_related))
  ads_annotated = ad_related_twts.ad_manual_adjusted.values
  adset = set(ads_annotated)
  print("unique ads:"+ str(len(adset)))
  ad_id_dict = {}
  i = 0
  for ad in adset : 
    if(getAdTweets(ad_related_twts, ad) >=2):
      ad_id_dict[ad] = i
      i = i+1
    else:
      print('ad with <2 samples: '+ str(ad))
  print(" No of ads with >=2 samples:"+ str(len(ad_id_dict)))
  ad_id_dict['none'] = len(ad_id_dict)
  print(ad_id_dict)
  return ad_id_dict

def convertAdNameToAdId(ad_id_dict, ad_name):
  if ad_name in ad_id_dict:
    return ad_id_dict[ad_name]
  else:
    return ad_id_dict['none']

ad_related_twts = get_ad_related_twts(df)

ad_id_dict = get_ad_id_dict(get_ad_related_twts(df))
ad_id_dict_len = len(ad_id_dict)
ad_id_dict['none'] = ad_id_dict_len
n_unique_ads = len(ad_id_dict)
df['ad_manual_adjusted_id'] = df['ad_manual_adjusted'].apply(lambda x: convertAdNameToAdId(ad_id_dict,x))


# ad related tweets: 3241
unique ads:80
ad with <2 samples: budweiser typical american
ad with <2 samples: ragged old flag
ad with <2 samples: fast & furious 9  trailer
ad with <2 samples: guitar hero
ad with <2 samples: bush's best
ad with <2 samples: bakari
ad with <2 samples: tide  ww
ad with <2 samples: discover card  no we don‚äôt charge annual fees
ad with <2 samples: secret
ad with <2 samples: fox  halftime show  teaser_3
ad with <2 samples: dashlane
ad with <2 samples: bernie the peoples perfume
ad with <2 samples: golden gronks
ad with <2 samples: ram trucks
ad with <2 samples: papa johns
ad with <2 samples: disney mulan
ad with <2 samples: discover card  yes we're accepted
ad with <2 samples: doritos the cool ranch
ad with <2 samples: patience
 No of ads with >=2 samples:61
{'new york life  love takes action': 0, 'fox nation  breaking news': 1, 'fox  super monday': 2, 'porsche  the heist': 3, 'marvel  black widow trailer': 4, 'budweiser  typical american': 5, 'amazon prime vi

In [ ]:
from sklearn.model_selection import train_test_split

# returns df with text_clean, ad column labelled as 0 0r 1
def get_bin_samples(df, colName, colVal):
  samples = pd.DataFrame()
  samples[colVal] = df[colName].apply(lambda ad: 1 if ad == colVal else 0)
  samples['text_clean'] = df['text_clean']
  return samples

# gives 50% negative ad samples, 50% positve ad samples
def get_ad_specific_sample_50_50(ad_labels):
  ad_positive_samples = ad_labels[ad_labels[ad] == 1]
  print(ad_positive_samples.shape)
  ad_negative_samples = ad_labels[ad_labels[ad] == 0]
  print(ad_negative_samples.shape)
  ad_sample = pd.DataFrame()

  if ad_positive_samples.shape[0]<= ad_negative_samples.shape[0]:
    ad_sample = pd.concat([ad_positive_samples, ad_negative_samples.iloc[0: ad_positive_samples.shape[0]]])
  else:
    ad_sample = pd.concat([ad_negative_samples, ad_positive_samples.iloc[0: ad_negative_samples.shape[0]]]) 
  print(ad_sample.shape)
  return ad_sample

def load_ad_annotations():
  ad_keywords = pd.read_csv('./SB_ad_annotations.csv')
  print(ad_keywords.columns)
  ad_keywords['keywords_clean'] = ad_keywords['Key Terms  Round 1'].apply(lambda x: x.lower()).apply(lambda x: cleanTweet(x))
  ad_keywords['Ad Name'] = ad_keywords['Ad Name'].apply(lambda x: x.lower())
  return ad_keywords

ad_keywords = load_ad_annotations()

Index(['Ad Number', 'Brand Name', 'Ad Name', 'Product', 'Key Terms  Round 1',
       'KeyTerms_Edited_1', 'KeyTerms_Edited', 'Excitatory Potential',
       'Emotional vs. Rational', 'Semantic Affinity', 'Valence', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')


Extract Embeddings, construct Siamese Network as mentioned in Sentence BERT to compute sentence similarity
Ref: https://github.com/prabhnoor0212/Siamese-Network-Text-Similarity/blob/master/quora_siamese.ipynb https://medium.com/@prabhnoor0212/siamese-network-keras-31a3a8f37d04
https://www.inovex.de/blog/transfer-learning-siamese-networks/


In [ ]:
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = n_unique_ads, # The number of output labels=2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# embeddings = model.get_input_embeddings()
# pretrain_embeddings = embeddings.weight.cpu().detach().numpy()
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
embedding_layer = model.get_input_embeddings()
print(embedding_layer)
pretrain_embeddings = embedding_layer.weight.cpu().detach().numpy()
print(pretrain_embeddings.shape)

Embedding(30522, 768, padding_idx=0)
(30522, 768)


In [ ]:
from keras.layers import LSTM

from keras.layers import Input, Concatenate, Flatten, Dense, Embedding, LSTM
from keras.models import Model
from keras.layers import Multiply, Dropout, Subtract
from keras.layers.core import Lambda
from keras.optimizers import Adam

# def get_lstm_model(input_sequences_1, input_sequences_2, embeddings, length_long_sentence =45):
def get_lstm_model(embeddings, length_long_sentence =45):

  #  input_1 and input_2 are pre-processed, Keras-tokenized text sequences which are to be compared
  # input_1 = Input(shape=(input_sequences_1.shape[1],))
  # input_2 = Input(shape=(input_sequences_2.shape[1],))
  input_1 = Input(shape=(length_long_sentence,))
  input_2 = Input(shape=(length_long_sentence,))
  # length_long_sentence = 45
  common_embed = Embedding(embeddings.shape[0], embeddings.shape[1], 
                                    weights = [embeddings], 
                                    input_length = length_long_sentence, trainable = False)
  lstm_1 = common_embed(input_1)
  lstm_2 = common_embed(input_2)

  common_lstm = LSTM(32, return_sequences = True, activation = 'relu')

  vector_1 = common_lstm(lstm_1)
  vector_1 = Flatten()(vector_1)

  vector_2 = common_lstm(lstm_2)
  vector_2 = Flatten()(vector_2)

  # difference of vectors
  x3 = Subtract()([vector_1, vector_2])

  x3 = Multiply()([x3, x3])

  x1_ = Multiply()([vector_1, vector_1])
  x2_ = Multiply()([vector_2, vector_2])

  # difference of squares
  x4 = Subtract()([x1_, x2_])
  
  # Lambda Layer to apply i.e cosine distance function before processing inputs
  x5 = Lambda(cosine_distance, output_shape = cos_dist_output_shape)([vector_1, vector_2])

  #concatnating, x^2-y^2, cosine(x,y), (x-y)^2
  # conc = Concatenate(axis=-1)([x5, x4, x3])
  conc = Concatenate(axis=-1)([vector_1, vector_2, x3])

  x = Dense(100, activation="relu", name='conc_layer')(conc)
  # x = Dense(100, activation="relu", name='conc_layer')(x3) # only using (x-y)^2 gave poor results
  x = Dropout(0.01)(x)
  out = Dense(1, activation="sigmoid", name = 'out')(x)

  lstm_model = Model([input_1, input_2], out)
  
  lstm_model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
  return lstm_model

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

def get_binary_pred_labels(predictions):
  pred_labels = []
  for pred in predictions:
    if pred[0]>=0.5 :
      pred_labels.append(1)
    else:
      pred_labels.append(0)
  return pred_labels

def get_binary_metrics(test_binary_labels, pred_labels, averageType = 'binary'):
  print("Evaluating metrics as per '"+averageType+"' average type")

  f1score = f1_score(test_binary_labels, pred_labels, average= averageType) # micro: Calculate metrics globally by counting the total true positives, false negatives and false positives.
  print('f1_score:'+ str(np.round(f1score,5)))

  prec = precision_score(test_binary_labels, pred_labels, average= averageType)
  print('Precision:'+ str(np.round(prec,5)))

  acc = accuracy_score(test_binary_labels, pred_labels)
  print("Accuracy: "+ str(np.round(acc,5)))

  recall = recall_score(test_binary_labels, pred_labels, average= averageType)
  print("recall: "+ str(np.round(recall,5)))

  confusionmatrix = confusion_matrix(test_binary_labels, pred_labels)
  if averageType == 'binary': 
    tn, fp, fn, tp = confusionmatrix.ravel()
    print("tp: "+ str(tp)+" tn: "+ str(tn)+" fp: "+ str(fp)+" fn: "+ str(fn))
    return f1score, prec, acc, recall, confusionmatrix, tp, tn, fp, fn
  else:
    return f1score, prec, acc, recall, confusionmatrix
# classification_report(true_labels_flat, pred_labels)


In [ ]:
# def get_tweets_train_test_val(df, ad):
#   ad_labels = get_bin_samples(df, 'ad_manual_adjusted', ad)
#   ad_specific_sample = get_ad_specific_sample_50_50(ad_labels)
#   total_sample_size = ad_specific_sample.shape[0]
#   twt_train, twt_test, label_train, label_test = train_test_split(ad_specific_sample['text_clean'], 
#                                                       ad_specific_sample[ad], test_size=0.2, random_state=42)

#   twt_train, twt_val, label_train, label_val = train_test_split(twt_train, 
#                                                       label_train, test_size=0.2, random_state=42)

#   train_tweets_padded = get_padded_Sentences(twt_train, tokenizer, length_long_sentence = 45)
#   val_tweets_padded = get_padded_Sentences(twt_val, tokenizer, length_long_sentence = 45)
#   test_tweets_padded = get_padded_Sentences(twt_test, tokenizer, length_long_sentence = 45)
#   return train_tweets_padded, test_tweets_padded, val_tweets_padded, label_train, label_test, label_val, total_sample_size

In [ ]:
def get_adkeys_train_test_val(ad, ad_keywords, total_sample_size):

  ad_keys = ad_keywords[ad_keywords['Ad Name'] == ad]['keywords_clean']

  # replicate ad keys to size of tweet sample so that cn be fed to the siamese network 
  ad_keys_replicated = ad_keys

  for i in range(total_sample_size-1):
    ad_keys_replicated = ad_keys_replicated.append(ad_keys)
  
  ad_padded_sentences = get_padded_Sentences(ad_keys_replicated, tokenizer, length_long_sentence = 45)
  ad_padded_train, ad_padded_test, _, _ = train_test_split(ad_padded_sentences, ad_padded_sentences,
                                              test_size=0.2, random_state=42)

  ad_padded_train, ad_padded_val, _, _ = train_test_split(ad_padded_train, ad_padded_train,
                                              test_size=0.2, random_state=42)

  return ad_padded_train, ad_padded_test, ad_padded_val

In [ ]:
import itertools
from random import sample
twt_train, twt_test, label_train, label_test = train_test_split(df['text_clean'], 
                                                      df['ad_manual_adjusted'], test_size=0.2, random_state=42)

train_set = pd.DataFrame()
train_set['text_clean'] = twt_train
train_set['ad_manual_adjusted'] = label_train
print(train_set.shape)
train_set = train_set.drop_duplicates()
print(train_set.shape)

tweets_left = []
tweets_right = []
target = []

ads = list(ad_id_dict.keys())

for ad in ads:
    # 1) create pairs of tweets talking about same ad, with a corresponding target of 1
    similar_tweets = train_set[train_set['ad_manual_adjusted'] == ad]['text_clean']
    similar_ad = ad_keywords[ad_keywords['Ad Name'] == ad]['keywords_clean'].values

    # Pick 1000 random pairs from the tweet combinations 
    group_pairs = list(itertools.combinations(similar_tweets, 2)) 
    positive_pairs = sample(group_pairs, 1000) if len(group_pairs) > 1000 else group_pairs
    tweets_left.extend([p[0] for p in positive_pairs])
    tweets_right.extend([p[1] for p in positive_pairs])
    # tweets_right.extend([similar_ad[0] for ind in positive_pairs])
    target.extend([1]*len(positive_pairs))
    
    # 2) create pairs of examples with tweets of different ads, with a target set to 0
    other_tweets = train_set[train_set['ad_manual_adjusted'] != ad]['text_clean']
    other_ads = ad_keywords[ad_keywords['Ad Name'] != ad]['keywords_clean']
    for i in range(len(positive_pairs)):
        tweets_left.append(np.random.choice(similar_tweets))
        tweets_right.append(np.random.choice(other_tweets))
        # tweets_right.append(np.random.choice(other_ads))
        target.append(0)

dataset = pd.DataFrame({
        'tweets_left': tweets_left,
        'tweets_right': tweets_right,
        'target': target
    }).sample(frac=1)  # Shuffle dataset
print(dataset.shape)
dataset.tail(100)

(5875, 2)
(4924, 2)
(43300, 3)


,tweets_left,tweets_right,target
24060,consequence watch sam elliott face off again l...,easily sam elliott and the doritos commercial,1
4095,that nfl ad with the kids moving right into li...,carlilloyd loved being included in the nfl sbl...,1
39050,im a big post malone fan,realsaavedra fact check donald trump is marrie...,0
14439,these alexa history ads are very careful to av...,what in the actual hell kyle going moochintamp...,0
3817,nfl turns out we needed more than one take jal...,that last commercial the nfl has a contest whe...,1
...,...,...,...
10567,i feel like im watching a strip show now super...,internet do your thing,0
21074,i guess if anyone is going to be in a michelob...,whole commercial with jimmy fallon talking to ...,1
42508,tough pill to swallow,nice try pepsi,0
6899,kysportsradio if i had just bought a hyundai t...,blitzvideos troy polamalu s coke zero commerci...,0


In [ ]:
dataset.to_csv('./siamese_train_dataset.csv')

In [ ]:
# pad and tokenize input tweets
tweets_left_padded = get_padded_Sentences(dataset['tweets_left'], tokenizer, length_long_sentence = 45)

In [ ]:
tweets_right_padded = get_padded_Sentences(dataset['tweets_right'], tokenizer, length_long_sentence = 45)

In [ ]:
print(dataset.shape)
x_pairs = [tweets_left_padded, tweets_right_padded]   # this will be the input of the siamese network
y = dataset['target'].values
print(len(y))
length_long_sentence = 45
lstm_model_1 = get_lstm_model(pretrain_embeddings, 45)
lstm_model_1.fit(x_pairs,
                 y, 
                 epochs = 5,
                 batch_size= 32
                ,validation_split=0.1
                 )

(43300, 3)
43300
Epoch 1/5
1218/1218 [==============================] - 154s 126ms/step - loss: 0.6892 - acc: 0.5491 - val_loss: 0.6879 - val_acc: 0.5312
Epoch 2/5
1218/1218 [==============================] - 153s 125ms/step - loss: 0.6837 - acc: 0.5550 - val_loss: 0.6850 - val_acc: 0.5494
Epoch 3/5
1218/1218 [==============================] - 152s 125ms/step - loss: 0.6768 - acc: 0.5693 - val_loss: 0.6744 - val_acc: 0.5788
Epoch 4/5
1218/1218 [==============================] - 153s 125ms/step - loss: 0.6587 - acc: 0.6144 - val_loss: 0.6462 - val_acc: 0.6291
Epoch 5/5
1218/1218 [==============================] - 153s 126ms/step - loss: 0.6157 - acc: 0.6759 - val_loss: 0.5982 - val_acc: 0.6885


In [ ]:
twt_test_padded = get_padded_Sentences(twt_test, tokenizer, length_long_sentence = 45)
print(len(twt_test_padded))
twt_train_padded = get_padded_Sentences(train_set['text_clean'], tokenizer, length_long_sentence = 45)

1469


In [ ]:
def get_prediction(lstm_model_1, twt_test_padded, twt_train_padded):
    """ Get the predicted ad category, and the most similar tweet
    in the train set."""
    # Compute similarities of test tweet with all tweets in the train set
    temp = np.array([twt_test_padded])
    twt_test_replicated = np.tile(temp, (len(twt_train_padded),1)) # replicate test tweet to size of train data

    similarities = lstm_model_1.predict([twt_test_replicated, twt_train_padded])
    # print(similarities)
    most_similar_index = np.argmax(similarities)
    max_sim = similarities[most_similar_index]
    
    # The predicted ad is the one of the most similar example from the train set
    prediction = train_set['ad_manual_adjusted'].iloc[most_similar_index]
    most_similar_train_example = train_set['text_clean'].iloc[most_similar_index]

    # top_n_indices = np.argsort(similarities.flatten())[::-1][:3]
    # for ind in top_n_indices:
    #   n_max_sim = similarities[ind]
    #   n_prediction = train_set['ad_manual_adjusted'].iloc[ind]
    #   n_most_similar_train_example = train_set['text_clean'].iloc[ind]
    #   print(n_max_sim)
    #   print("similar ad:"+ n_prediction)
    #   print("similar sentence: "+ n_most_similar_train_example)

    return prediction, most_similar_train_example, max_sim
    # return pd.Series([prediction,most_similar_train_example,max_sim],index=['prediction','most_similar_train_example','max_sim'])
    
print('len of train samples: '+ str(len(twt_train_padded)))

len of train samples: 4924


In [ ]:
predictions_siamese = []
pred_most_sim_example = []
pred_max_sim = []
print("len of test samples: "+ str(len(twt_test_padded)))
# print(twt_test_df.shape)
i = 0

# to check for overfitting, do minor variations in tweet and check for max similar ad 

# test = ['jonschweppe google easily best far even close', 'jonschweppe google best easily far even close'
# , 'jonschweppe google easily best far close', 
# 'jonschweppe google easily best far even',
# 'jonschweppe google easily best far']

# twt_test_padded = get_padded_Sentences(test, tokenizer, length_long_sentence = 45)

for test_padded_sent in twt_test_padded:
  print(i)
  # print("input sentence: "+ test[i])
  pred, most_sim_example, max_sim = get_prediction(lstm_model_1, test_padded_sent, twt_train_padded)
  predictions_siamese.append(pred)
  pred_most_sim_example.append(most_sim_example)
  pred_max_sim.append(max_sim)
  # print('-----------')
  
  i = i+1


In [ ]:
# using 'weighted' averageType since multiple ads are present, if binary use avgType as 'binary'

predictions_df = pd.DataFrame()
predictions_df['text_clean'] = twt_test
predictions_df['predictions'] = predictions_siamese
predictions_df['most_sim_example'] = pred_most_sim_example
predictions_df['max_sim'] = pred_max_sim
predictions_df.to_csv('./predictions_siamese.csv')
f1score, prec, acc, recall, conf_matrix = get_binary_metrics(label_test, predictions_siamese, 'weighted')


##################### Siamese Network code ends here #######################

In [ ]:
# print(dataset.shape)
# x_pairs = [tweets_left, tweets_right]   # this will be the input of the siamese network

# y = dataset['target'].values
# length_long_sentence = 45
# lstm_model_1 = get_lstm_model(pretrain_embeddings, 45)
# lstm_model_1.fit(x_pairs,
#                  y, 
#                  epochs = 5,
#                  batch_size= 32
#                 #  , validation_data=([val_tweets_padded, ad_padded_val],
#                 #             label_val)
#                  )
# # predictions_siamese = lstm_model.predict([test_tweets_padded, ad_padded_test])
# # pred_labels_siamese = get_binary_pred_labels(predictions_siamese)


In [ ]:
from keras import backend as K

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
import pandas as pd
import numpy as np

res_cols = ['ad', 'ad_related_size', 'train_size', 'test_size', 'f1_score', 'precision', 'accuracy', 'recall' , 'tp', 'tn', 'fp' 'fn']
# res_cols = ['ad', 'ad_related_size', 'train_size', 'test_size', 'f1_score', 'precision', 'accuracy', 'recall', 'tp', 'tn', 'fp', 'fn' ]
siamese_results = pd.DataFrame(columns = res_cols )
ads = []
ad_related_sizes = []
train_sizes = []
test_sizes = []
f1_scores = []
precisions = []
accuracies = []
recalls = []
tps = []
tns = []
fps = []
fns = []

ads_list = list(ad_id_dict.keys())
ads_list.remove('none')
for ad in ads_list:
  train_tweets_padded, test_tweets_padded, val_tweets_padded, label_train, label_test, label_val, total_sample_size= get_tweets_train_test_val(df, ad)
  if total_sample_size<= 25:
    continue
  ad_padded_train, ad_padded_test, ad_padded_val = get_adkeys_train_test_val(ad, ad_keywords, total_sample_size)

  lstm_model = get_lstm_model(train_tweets_padded, ad_padded_train, pretrain_embeddings)
  lstm_model.fit([train_tweets_padded, ad_padded_train],
                  label_train, epochs = 5,
                  batch_size=32, 
                  validation_data=([val_tweets_padded, ad_padded_val],
                            label_val))
  predictions_siamese = lstm_model.predict([test_tweets_padded, ad_padded_test])
  pred_labels_siamese = get_binary_pred_labels(predictions_siamese)
  
  train_size = ad_padded_train.shape[0]
  test_size = ad_padded_test.shape[0]
  ad_related_size = train_size + test_size + val_tweets_padded.shape[0]

  # Save a trained model, configuration and tokenizer using `save_pretrained()`. # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  temp_output_dir = output_dir+"/"+ ad+"/"
  if not os.path.exists(temp_output_dir):
    os.makedirs(temp_output_dir)
  model_to_save.save_pretrained(temp_output_dir)

  f1score, prec, acc, recall, _, tp, tn, fp, fn = get_binary_metrics(label_test, pred_labels_siamese)
  # data = np.array([ad, ad_related_size, train_size, test_size, f1score,prec, acc,recall])
  # curr_result = pd.Series( data, index = res_cols)
  # siamese_results = siamese_results.append(curr_result, ignore_index=True)
  ads.append(ad)
  ad_related_sizes.append(ad_related_size)
  train_sizes.append(train_size)
  test_sizes.append(test_size)
  f1_scores.append(f1score)
  precisions.append(prec)
  accuracies.append(acc)
  recalls.append(recall)
  tps.append(tp)
  tns.append(tn)
  fps.append(fp)
  fns.append(fn)

siamese_results['ad'] = ads
siamese_results['ad_related_size'] = ad_related_sizes
siamese_results['train_size'] = train_sizes
siamese_results['test_size'] = test_sizes
siamese_results['f1_score'] = f1_scores
siamese_results['precision'] = precisions
siamese_results['accuracy'] = accuracies
siamese_results['recall'] = recalls
siamese_results['tp'] = tps
siamese_results['tn'] = tns
siamese_results['fp'] = fps
siamese_results['fn'] = fns

siamese_results.head(5)

In [ ]:
siamese_results.to_csv('./siamese_results.csv')